In [1]:
!brew install tesseract

==> Auto-updating Homebrew...
Adjust how often this is run with HOMEBREW_AUTO_UPDATE_SECS or disable with
HOMEBREW_NO_AUTO_UPDATE. Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).
Error: Failed to download https://formulae.brew.sh/api/formula.jws.json!
Failed to download https://formulae.brew.sh/api/cask.jws.json!
Failed to download https://formulae.brew.sh/api/formula_tap_migrations.jws.json!
Failed to download https://formulae.brew.sh/api/cask_tap_migrations.jws.json!
==> Downloading https://formulae.brew.sh/api/formula.jws.json
==> Downloading https://formulae.brew.sh/api/formula_tap_migrations.jws.json
==> Downloading https://formulae.brew.sh/api/cask.jws.json
To reinstall 5.5.1, run:
  brew reinstall tesseract


In [2]:
!brew install pyenv
!pyenv install 3.10.12
!pyenv global 3.10.12

To reinstall 2.6.4, run:
  brew reinstall pyenv
pyenv: /Users/macbookpro/.pyenv/versions/3.10.12 already exists
^Cntinue with installation? (y/N) 


In [3]:
!pip install numpy==1.26.4
!pip install torch==2.1.2 torchvision==0.16.2
!pip install ultralytics

ERROR: Could not find a version that satisfies the requirement torch==2.1.2 (from versions: none)
ERROR: No matching distribution found for torch==2.1.2


In [4]:
!pip install pytesseract

In [1]:
from ultralytics import YOLO

In [2]:
import os
import xml.etree.ElementTree as ET

# === Configuration ===
XML_PATH = "annotations.xml"  # Path to your CVAT-style XML
IMAGE_FOLDER = "images"       # Folder where your images are stored
OUTPUT_LABEL_FOLDER = "labels"  # Where YOLO annotation .txt files will be saved

# Define class names and assign IDs
CLASSES = [
    "passport", "photo", "name", "surname", "date_of_birth", "sex",
    "mrz", "number", "other", "nationality"
]
CLASS_ID = {name: i for i, name in enumerate(CLASSES)}

# === Create output folder ===
os.makedirs(OUTPUT_LABEL_FOLDER, exist_ok=True)

# === Parse XML ===
tree = ET.parse(XML_PATH)
root = tree.getroot()

# === Iterate over each image ===
for image in root.findall("image"):
    filename = os.path.basename(image.get("name"))
    img_width = float(image.get("width"))
    img_height = float(image.get("height"))
    
    txt_filename = os.path.splitext(filename)[0] + ".txt"
    txt_path = os.path.join(OUTPUT_LABEL_FOLDER, txt_filename)

    yolo_lines = []

    # Process bounding boxes
    for box in image.findall("box"):
        label = box.get("label")
        if label not in CLASS_ID:
            continue  # skip unknown labels

        class_id = CLASS_ID[label]

        xtl = float(box.get("xtl"))
        ytl = float(box.get("ytl"))
        xbr = float(box.get("xbr"))
        ybr = float(box.get("ybr"))

        # Convert to YOLO format (normalized)
        x_center = ((xtl + xbr) / 2) / img_width
        y_center = ((ytl + ybr) / 2) / img_height
        width = (xbr - xtl) / img_width
        height = (ybr - ytl) / img_height

        yolo_line = f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
        yolo_lines.append(yolo_line)

    # Save to YOLO txt file
    with open(txt_path, "w") as f:
        f.write("\n".join(yolo_lines))

print("✅ YOLO label files saved in:", OUTPUT_LABEL_FOLDER)


✅ YOLO label files saved in: labels


In [3]:
import numpy, torch

In [12]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
model.train(data="data.yaml", epochs=50, imgsz=640, batch=16)

New https://pypi.org/project/ultralytics/8.3.170 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.163 🚀 Python-3.12.6 torch-2.2.2 CPU (Intel Core(TM) i7-9750H 2.60GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train17, nbs=64, nms=False, opset=None, optimize=False, optimizer=au

train: Scanning /Users/macbookpro/Downloads/golomt bank intern/algorithms and co

train: New cache created: /Users/macbookpro/Downloads/golomt bank intern/algorithms and codes/dataset/labels/train.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1524.0±442.8 MB/s, size: 2710.6 KB)



val: Scanning /Users/macbookpro/Downloads/golomt bank intern/algorithms and code

val: New cache created: /Users/macbookpro/Downloads/golomt bank intern/algorithms and codes/dataset/labels/val.cache
Plotting labels to runs/detect/train17/labels.jpg... 


...

                   all          5        170      0.733      0.258      0.321       0.19

50 epochs completed in 0.112 hours.


Optimizer stripped from runs/detect/train17/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train17/weights/best.pt, 6.2MB

Validating runs/detect/train17/weights/best.pt...
Ultralytics 8.3.163 🚀 Python-3.12.6 torch-2.2.2 CPU (Intel Core(TM) i7-9750H 2.60GHz)
Model summary (fused): 72 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  m


                   all          5        170      0.732      0.258      0.318      0.189
                  name          5         10          1          0          0          0
               surname          5         10          1          0       0.11     0.0836
         date_of_birth          5         10          1          0      0.269      0.179
                   sex          5         10          0          0          0          0
                   mrz          5         10      0.399          1      0.773      0.331
                number          5         10          1          0          0          0
                 other          5        100      0.694      0.567       0.72      0.507
           nationality          5         10      0.766        0.5      0.672      0.415
Speed: 2.6ms preprocess, 62.2ms inference, 0.0ms loss, 14.3ms postprocess per image
Results saved to runs/detect/train17


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([2, 3, 4, 5, 6, 7, 8, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x15926bbf0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
  

In [13]:
model = YOLO("runs/detect/train17/weights/best.pt")

In [15]:
results = model("dataset/images/val/21.jpg")[0]


image 1/1 /Users/macbookpro/Downloads/golomt bank intern/algorithms and codes/dataset/images/val/21.jpg: 640x480 3 mrzs, 11 others, 63.9ms
Speed: 2.7ms preprocess, 63.9ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 480)


In [16]:
import pytesseract
from PIL import Image

In [18]:
image = Image.open("dataset/images/val/21.jpg")

for box in results.boxes:
    class_id = int(box.cls[0])
    label = model.names[class_id]
    x1, y1, x2, y2 = map(int, box.xyxy[0])

    # Crop the field
    field = image.crop((x1, y1, x2, y2))

    # OCR the cropped field
    text = pytesseract.image_to_string(field, config="--psm 6")
    print(f"{label}: {text.strip()}")

other: 
other: PASAPORTE
other: PASSPORT
other: PASSEPORT
mrz: PSUSAGORDONCSKSCSTEVESS SSS SOKO OOOO OOS SE
other: Jatée’ of expiration / Date d'expiration/ Fecha de caducidad
other: SEE PAGE 27
other: Vepartment of Stal
other: Date of birth / Date de naissance / Fecha de nacimiento
other: Jnited-States.
mrz: PSUS AGOQORDONXKKSTEVECCS KEKE eee eS
mrz: RDONCSSST EVESSS SOOO KOO SOOKE OOS EOS
other: INITED STATES OF AMERICA
other: “Jace of birth / Lieu de naissance / Lugar de nacimiento
